## Prompts Evaluation with Prompt Management & Prompt Flows

In this example, we'll explore how to evaluate prompts using a combination of Prompt Management and Prompt Flows in Amazon Bedrock.

Evaluating prompts is an essential step in the prompt lifecycle. Using LLM-as-a-judge for validating your prompts according to your own criteria allows you to efficiently quantify the prompts quality for optimization at scale.


<img src="./images/prompt_eval_diagram.png" width="30%">


[Amazon Bedrock Prompt Management](https://docs.aws.amazon.com/bedrock/latest/userguide/prompt-management.html) streamlines the creation, evaluation, deployment, and sharing of prompts in the Amazon Bedrock console and via APIs in the SDK. This feature helps developers and business users obtain the best responses from foundation models for their specific use cases.

[Amazon Bedrock Prompt Flows](https://docs.aws.amazon.com/bedrock/latest/userguide/flows.html) allows you to easily link multiple foundation models (FMs), prompts, and other AWS services, reducing development time and effort. It introduces a visual builder in the Amazon Bedrock console and a new set of APIs in the SDK, that simplifies the creation of complex generative AI workflows.

Let's start by making sure we have the lastest version of the Amazon Bedrock SDK, importing the libraries, and setting-up the client.

In [ ]:
# Only run this the first time...
!pip3 install boto3 botocore matplotlib -qU

In [ ]:
import boto3
from datetime import datetime
import json

Note the Prompt Management and Flows features are part of the Bedrock Agent SDK.

In [ ]:
### Adjust with your preferred region accordingly:
region = "us-east-1"

bedrock_agent = boto3.client(service_name = "bedrock-agent", region_name = region)

### Adjust with your preferred model IDs for invocations and evaluation - Note some models are only available in certain regions:
modelInvokeId = "amazon.titan-text-premier-v1:0"
modelEvalId = "anthropic.claude-3-sonnet-20240229-v1:0"


### Evaluation Prompt

Let's create our sample evaluation prompt by leveraging on Prompt Management for Amazon Bedrock. Here, you can adjust the sample prompt template and evaluation criteria provided according to your specific needs.

In [ ]:
response = bedrock_agent.create_prompt(
    name = f"prompt-evaluator",
    description = "Prompt template for evaluating prompt responses with LLM-as-a-judge",
    variants = [
        {
            "inferenceConfiguration": {
            "text": {
                "maxTokens": 2000,
                "temperature": 0,
            }
            },
            "modelId": modelEvalId,
            "name": "variantOne",
            "templateConfiguration": {
                "text": {
                    "inputVariables": [
                        {
                            "name": "input"
                        },
                        {
                            "name": "output"
                        }
                    ],
                    "text": """
You're an evaluator for the prompts and answers provided by a generative AI model. Consider the input prompt in \
the <input> tags, the output answer in the <output> tags, the prompt evaluation criteria in the <prompt_criteria> tags, \
and the answer evaluation criteria in the <answer_criteria> tags.

<input>
{{input}}
</input>

<output>
{{output}}
</output>

<prompt_criteria>
- The prompt should be clear, direct, and detailed.
- The question, task, or goal should be well explained and be grammatically correct.
- The prompt is better if containing examples.
- The prompt is better if specifies a role or sets a context.
- The prompt is better if provides details about the format and tone of the expected answer.
</prompt_criteria>

<answer_criteria>
- The answers should be correct, well structured, and technically complete.
- The answers should not have any hallucinations, made up content, or toxic content.
- The answer should be grammatically correct.
- The answer should be fully aligned with the question or instruction in the prompt.
</answer_criteria>

Evaluate the answer the generative AI model provided in the <output> with a score from 0 to 100 according to the <answer_criteria> provided; \
any hallucinations, even if small, should dramatically impact the evaluation score.
Also evaluate the prompt passed to that generative AI model provided in the <input> with a score from 0 to 100 according to the <prompt_criteria> provided.
Respond only with a JSON having:
- An 'answer-score' key with the score number you evaluated the answer with.
- A 'prompt-score' key with the score number you evaluated the prompt with.
- A 'justification' key with a justification for the two evaluations you provided to the answer and the prompt; make sure to explicitely include any errors or hallucinations in this part.
- An 'input' key with the content of the <input> tags.
- An 'output' key with the content of the <output> tags.
- A 'prompt-recommendations' key with recommendations for improving the prompt based on the evaluations performed.
Skip any preamble or any other text apart from the JSON in your answer.
                    """
                }
            },
            "templateType": "TEXT"
        }
    ],
    defaultVariant = "variantOne"
)
print(json.dumps(response, indent=2, default=str))
promptEvalId = response["id"]
promptEvalArn = response["arn"]
promptEvalName = response["name"]
print(f"Prompt ID: {promptEvalId}\nPrompt ARN: {promptEvalArn}\nPrompt Name: {promptEvalName}")

Now that we have a draft prompt, we can create a version from it.

In [ ]:
response = bedrock_agent.create_prompt_version(
    promptIdentifier = promptEvalId
)
print(json.dumps(response, indent=2, default=str))

### Evaluation Flow

Now that we have our evaluation prompt, we can work on a workflow for running the evaluations with it. For this we'll rely on Prompt Flows for Amazon Bedrock.

Let's create a simple flow that will invoke a given model with our input prompts for obtaining the outputs or responses, and then load the evaluation prompt from our catalog for obtaining the evaluation score.

<img src="./images/prompt_eval_flow.png" width="60%">


We'll need an AWS IAM role for creating the Prompt Flow in Amazon Bedrock. If you already have a role with your permissions you can directly replace the ```flowRole``` variable with your role's ARN.

For simplicity in this example we'll create a new role and attach the ```AmazonBedrockFullAccess``` policy to it. In general, it's recommended that you further limit the policies with conditions.

You can check further details in the [How Prompt Flows for Amazon Bedrock works](https://docs.aws.amazon.com/bedrock/latest/userguide/flows-how-it-works.html) documentation.

In [ ]:
### Create a new AWS IAM role for this example (comment this section if using your own role)
iam = boto3.client('iam')
response = iam.create_role(
    RoleName = 'MyBedrockFlowsRole',
    AssumeRolePolicyDocument = json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "bedrock.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    })
)
flowRole = response['Role']['Arn']
response = iam.attach_role_policy(
    RoleName = 'MyBedrockFlowsRole',
    PolicyArn = 'arn:aws:iam::aws:policy/AmazonBedrockFullAccess'
)

### Or specify your own AWS IAM role (uncomment and replace the ARN if using your own role)
#flowRole = 'YOUR-IAM-ROLE-ARN'

print(f'Using flowRole: {flowRole}')

In [ ]:
response = bedrock_agent.create_flow(
    name = f"prompt-eval-flow",
    description = "Prompt Flow for evaluating prompts with LLM-as-a-judge.",
    executionRoleArn = flowRole,
    definition = {
        "nodes": [
          {
            "name": "Start",
            "type": "Input",
            "configuration": {
              "input": {}
            },
            "outputs": [
              {
                "name": "document",
                "type": "String"
              }
            ],
          },
          {
            "name": "End",
            "type": "Output",
            "configuration": {
              "output": {}
            },
            "inputs": [
              {
                "expression": "$.data",
                "name": "document",
                "type": "String"
              }
            ],
          },
          {
            "name": "Invoke",
            "type": "Prompt",
            "configuration": {
              "prompt": {
                "sourceConfiguration": {
                  "inline": {
                    "inferenceConfiguration": {
                      "text": {
                        "maxTokens": 2000,
                        "temperature": 0,
                      }
                    },
                    "modelId": modelInvokeId,
                    "templateConfiguration": {
                      "text": {
                        "inputVariables": [
                          {
                            "name": "input"
                          }
                        ],
                        "text": "{{input}}"
                      }
                    },
                    "templateType": "TEXT"
                  }
                }
              }
            },
            "inputs": [
              {
                "expression": "$.data",
                "name": "input",
                "type": "String"
              }
            ],
            "outputs": [
              {
                "name": "modelCompletion",
                "type": "String"
              }
            ],
          },
          {
            "name": "Evaluate",
            "type": "Prompt",
            "configuration": {
              "prompt": {
                "sourceConfiguration": {
                  "resource": {
                    "promptArn": promptEvalArn
                  }
                }
              }
            },
            "inputs": [
              {
                "expression": "$.data",
                "name": "input",
                "type": "String"
              },
              {
                "expression": "$.data",
                "name": "output",
                "type": "String"
              }
            ],
            "outputs": [
              {
                "name": "modelCompletion",
                "type": "String"
              }
            ],
          }
        ],
        "connections": [
          {
            "name": "StartToInvoke",
            "source": "Start",
            "target": "Invoke",
            "type": "Data",
            "configuration": {
              "data": {
                "sourceOutput": "document",
                "targetInput": "input"
              }
            },
          },
          {
            "name": "InvokeToEvaluate",
            "source": "Invoke",
            "target": "Evaluate",
            "type": "Data",
            "configuration": {
              "data": {
                "sourceOutput": "modelCompletion",
                "targetInput": "output"
              }
            },
          },
          {
            "name": "StartToEvaluate",
            "source": "Start",
            "target": "Evaluate",
            "type": "Data",
            "configuration": {
              "data": {
                "sourceOutput": "document",
                "targetInput": "input"
              }
            },
          },
          {
            "name": "EvaluateToEnd",
            "source": "Evaluate",
            "target": "End",
            "type": "Data",
            "configuration": {
              "data": {
                "sourceOutput": "modelCompletion",
                "targetInput": "document"
              }
            },
          }
        ]
    }
)
print(json.dumps(response, indent=2, default=str))
flowEvalId = response["id"]
flowEvalArn = response["arn"]
flowEvalName = response["name"]
print(f"Flow ID: {flowEvalId}\nFlow ARN: {flowEvalArn}\nFlow Name: {flowEvalName}")

Now that we have our first flow, we can prepare it. This basically builds and validates our flow.

In [ ]:
response = bedrock_agent.prepare_flow(
    flowIdentifier = flowEvalId
)
print(json.dumps(response, indent=2, default=str))

We can now get the flow to double-check the "status" is "prepared", otherwise go back to the previous steps for solving any issues.

In [ ]:
response = bedrock_agent.get_flow(
    flowIdentifier = flowEvalId
)
print("Status:", response["status"])

Let's create a version from our draft flow. Note flow versions are read-only, meaning these cannot be modified once created as they're intended for using in production. If you need to make changes to a flow you can update your draft.

In [ ]:
response = bedrock_agent.create_flow_version(
    flowIdentifier = flowEvalId
)
print(json.dumps(response, indent=2, default=str))

We can also create flow alises, so that we can point our application front-ends and any other integrations to these. This allows creating new versions without impacting our service.

In [ ]:
response = bedrock_agent.create_flow_alias(
    flowIdentifier = flowEvalId,
    name = flowEvalName,
    description = "Alias for my prompt evaluation flow",
    routingConfiguration = [
        {
            "flowVersion": "1"
        }
    ]
)
print(json.dumps(response, indent=2, default=str))
flowEvalAliasId = response['id']

### Testing the Evaluation Flow

Now that we have our prompt evaluation flow, we can test it with a few invocations. For this we'll rely on the Bedrock Agent Runtime SDK.

You can invoke flows from any application front-end or your own systems as required. It effectively exposes all the logic of your flow through an Agent Endpoint API.

In [ ]:
bedrock_agent_runtime = boto3.client(service_name = 'bedrock-agent-runtime', region_name = region)

Let's create a handy function for running the evaluation against a given prompt...

In [ ]:
def evaluatePrompt(prompt):
    response = bedrock_agent_runtime.invoke_flow(
        flowIdentifier = flowEvalId,
        flowAliasIdentifier = flowEvalAliasId,
        inputs = [
            { 
                "content": { 
                    "document": prompt
                },
                "nodeName": "Start",
                "nodeOutputName": "document"
            }
        ]
    )
    event_stream = response["responseStream"]
    for event in event_stream:
        #print(json.dumps(event, indent=2, ensure_ascii=False))
        if "flowOutputEvent" in event:
            evalResponse = json.loads(event["flowOutputEvent"]["content"]["document"])
    if evalResponse:
        evalResponse["modelInvoke"] = modelInvokeId
        evalResponse["modelEval"] = modelEvalId
        #print(json.dumps(evalResponse, indent=2, default=str))
        return evalResponse

We can test with a sample prompt, and visualize the results...

In [ ]:
evaluatePrompt("What is cloud computing in a single paragraph?")

----------

### Prompt Evaluation at Scale

Now that we have a flow that is able to evaluate our prompt, we can programmatically extend this for running against a full dataset of prompts at scale. For this, you can either leverage on a dataset stored in an Amazon S3 bucket, or load a file locally like we do in this example notebook.

In the same way, you can write the results in another file in Amazon S3, or visualize it locally like we do in this example notebook.

For our example, let's load a sample dataset with 4 simple prompts...

In [ ]:
# Read prompts dataset file locally

import json
promptsDataset = []
with open('prompts_dataset.jsonl') as f:
    for line in f:
        promptsDataset.append(json.loads(line))

if promptsDataset:
    for i, j in enumerate(promptsDataset):
        print(f'{i+1}. {j["input"]}')

We can now evaluate each prompt in our dataset with the function we created before...

In [ ]:
from datetime import datetime
if promptsDataset:
    results = []
    for i, j in enumerate(promptsDataset):
        print(f"{datetime.now().strftime('%H:%M:%S')} - Evaluating prompt {i+1} of {len(promptsDataset)}...")
        try:
            results.append(evaluatePrompt(j["input"]))
        except Exception as e:
            print(f"Error evaluating prompt {i+1}: {e}")
            results.append({"error": str(e)})
    print("All prompts evaluated.")

Let's review the results of the evaluations...

In [ ]:
for i in results:
    print(json.dumps(i, indent=2, ensure_ascii=False))

We can also create a graph with the scores...

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract the scores from the data
scores = [result['prompt-score'] for result in results]

# Create a list of labels for the x-axis
labels = [f"Prompt {i+1}" for i in range(len(scores))]

# Set up the figure and axis
fig, ax = plt.subplots(figsize=(8, 4))

# Create the bar plot
ax.bar(labels, scores)

# Set the title and axis labels
ax.set_title("Evaluation Scores", fontsize=12)
ax.set_xlabel("Prompts", fontsize=10)
ax.set_ylabel("Score", fontsize=10)

# Rotate the x-axis labels for better visibility
plt.xticks(rotation=45, fontsize=8)

# Add grid lines
ax.grid(axis='y', linestyle='--')

# Adjust the spacing between subplots
plt.subplots_adjust(bottom=0.3)

# Add a horizontal line at score 80
ax.axhline(y=80, color='r', linestyle='--', label='Passing threshold')

# Add a legend
ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1))

# Show the plot
plt.show()

Note that, in the example above, we have iterated manually through our dataset using a Python loop writing to a list.

Alternatively, you might want to orchestrate this iteration using your own CI/CD pipeline, or even relying on the ```Iterator```, ```Collector```, ```S3 Retrieval``` and ```S3 Storage``` nodes  in Prompt Flows for Amazon Bedrock as per the following sample flow.

<img src="./images/prompt_eval_flow_scale.png" width="80%">

These alternative methods are out of the scope of this example.

---

### Cleaning-up Resources (optional)

Before leaving, here's how to delete the resources that we've created.

In [ ]:
response = bedrock_agent.delete_flow_alias(
    flowIdentifier = flowEvalId,
    aliasIdentifier = flowEvalAliasId
)
print(json.dumps(response, indent=2, default=str))

In [ ]:
response = bedrock_agent.delete_flow_version(
    flowIdentifier = flowEvalId,
    flowVersion = '1'
)
print(json.dumps(response, indent=2, default=str))

In [ ]:
response = bedrock_agent.delete_flow(
    flowIdentifier = flowEvalId
)
print(json.dumps(response, indent=2, default=str))

In [ ]:
response = bedrock_agent.delete_prompt(
    promptIdentifier = promptEvalId
)
print(json.dumps(response, indent=2, default=str))

In [ ]:
response = iam.detach_role_policy(
    RoleName='MyBedrockFlowsRole',
    PolicyArn='arn:aws:iam::aws:policy/AmazonBedrockFullAccess'
)
print(json.dumps(response, indent=2, default=str))

response = iam.delete_role(
    RoleName = 'MyBedrockFlowsRole'
)
print(json.dumps(response, indent=2, default=str))

-------